<a href="https://colab.research.google.com/github/Aditya-jr/NLP/blob/main/NLP_fakenews_classifr_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dataset= https://www.kaggle.com/competitions/fake-news/data  \
**Dataset Description**\
train.csv: A full training dataset with the following attributes:

id: unique id for a news article\
title: the title of a news article\
author: author of the news article\
text: the text of the article; could be incomplete\
label: a label that marks the article as potentially unreliable\
1: unreliable\
0: reliable\

In [5]:
## fake news classifier using bidirectional LSTM

In [6]:
import pandas as pd
import numpy as np


In [7]:
df= pd.read_csv('train.csv')
#engine='python',error_bad_lines=False)

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df =df.dropna()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [14]:
df.shape

(18285, 5)

In [15]:
## get independent features
X=df.drop('label',axis=1)

In [16]:
X.head(2)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...


In [17]:
#dependent variable
y=df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [18]:
#check balance/imbalalce dataset or no
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'2.9.2'

In [21]:
from tensorflow.keras.layers import Embedding

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [23]:
## vocabulary size
voc_size =5000

In [24]:
messages= X.copy()

In [25]:
messages.shape

(18285, 4)

In [26]:
messages.reset_index(inplace=True)

In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
## datapreprocessing

In [30]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [31]:
corpus = []

for i in range(0,len(messages)):
  #print(i)
  review = re.sub('[^a-zA-Z]', ' ',messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)


In [32]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [33]:
# one hot 

onehot_represnt = [one_hot(words,voc_size) for words in corpus]

In [34]:
print(onehot_represnt)

[[2431, 2093, 1132, 2777, 633, 1826, 3192, 3541, 2014, 689], [1490, 3864, 1098, 334, 4558, 3247, 683], [2087, 2393, 2418, 4443], [4400, 1919, 3259, 1953, 341, 2784], [3524, 4558, 2074, 4418, 791, 165, 4558, 2778, 2401, 924], [3348, 2679, 2402, 494, 180, 496, 2857, 4489, 309, 1919, 1186, 246, 769, 317, 683], [457, 4936, 2153, 4436, 2492, 2459, 1743, 436, 2806, 3121, 3712], [2709, 631, 3220, 4927, 2230, 3533, 496, 181, 2806, 3121, 3712], [2898, 2137, 239, 4273, 320, 2751, 2254, 594, 496, 3797], [4988, 3956, 436, 2927, 3116, 2749, 653, 4188], [3928, 144, 4798, 2705, 1080, 1090, 2552, 4649, 4528, 2621, 1347], [1953, 1431, 633, 2751, 496, 2230], [2763, 2712, 3451, 2504, 1058, 2388, 2818, 943, 4634], [2335, 4099, 4713, 2726, 1737, 2609, 3234, 2806, 3121, 3712], [1413, 3414, 1413, 4149, 2292, 2806, 3121, 3712], [597, 432, 4205, 3664, 282, 3946, 1260, 4018, 3169, 3009], [2398, 3586, 3864], [4158, 2837, 102, 2743, 496, 3021, 2383, 683], [1661, 2689, 1098, 3301, 2546, 3892, 4799, 4065, 165], [24

In [36]:
# pass to embedding layer and then to bidirectional LSTM

Embedding representation

In [37]:
sent_length = 20 #max sentence length
embedded_doc=pad_sequences(onehot_represnt,padding='pre',maxlen=sent_length)
embedded_doc

array([[   0,    0,    0, ..., 3541, 2014,  689],
       [   0,    0,    0, ..., 4558, 3247,  683],
       [   0,    0,    0, ..., 2393, 2418, 4443],
       ...,
       [   0,    0,    0, ..., 2806, 3121, 3712],
       [   0,    0,    0, ...,  818, 1017, 1951],
       [   0,    0,    0, ..., 1825, 3129,  496]], dtype=int32)

In [38]:
from unicodedata import bidirectional
## creating model
embedding_vector_features = 40
model= Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
X_final = np.array(embedded_doc)
y_final=np.array(y)

In [42]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X_final, y_final, test_size=0.33, random_state=42) 

## model training


In [46]:
model.fit(X_train,y_train,validation_data=(X_test ,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0058 - accuracy: 0.9988 - val_loss: 0.6005 - val_accuracy: 0.9034
Epoch 2/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0069 - accuracy: 0.9980 - val_loss: 0.6208 - val_accuracy: 0.9065
Epoch 3/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0049 - accuracy: 0.9986 - val_loss: 0.6464 - val_accuracy: 0.9012
Epoch 4/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.6860 - val_accuracy: 0.9074
Epoch 5/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0076 - accuracy: 0.9974 - val_loss: 0.5527 - val_accuracy: 0.9074
Epoch 6/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.6454 - val_accuracy: 0.9074
Epoch 7/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0029 - accuracy: 0.9988 - val_loss: 0.6903 - val_accuracy: 0.9064
Epoch 

In [47]:
## cofusion matrix for prediction

In [48]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [49]:

y_pred=np.where(y_pred>=.5,1,0)

In [50]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [51]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)


array([[3088,  331],
       [ 218, 2398]])

In [52]:
accuracy_score(y_test,y_pred)

0.9090306545153273